In [1]:
import pgmpy

# How to define a random variable, its values, and its distribution?

In pgmpy,
> The workflow is to first define the model structure, then define all the parameters (CPDs) and then add these parameters to the model.

The BayesianModel can be initialized by edges in the model structure. For example, the code below defines a model with edges:
* Pollution --> Cancer
* Smoker --> Cancer
* Cancer --> Xray
* Cancer --> Dyspnoea

In [1]:
from pgmpy.models import BayesianModel

cancer_model = BayesianModel([["Pollution", "Cancer"],
                             ["Smoker", "Cancer"],
                             ["Cancer", "Xray"],
                             ["Cancer", "Dyspnoea"]])

In [5]:
# Variables
cancer_model.nodes

NodeView(('Pollution', 'Cancer', 'Smoker', 'Xray', 'Caner', 'Dyspnoea'))

In [7]:
# Structure (i.e. edges)
cancer_model.edges

OutEdgeView([('Pollution', 'Cancer'), ('Cancer', 'Xray'), ('Smoker', 'Cancer'), ('Caner', 'Dyspnoea')])

Specifying the conditional probability distribution.
In the model above, how many CPDs do we need to specify?
* Pr(Pollution)
* Pr(Smoker)
* Pr(Cancer | Pollution, Smoker)
* Pr(Xray | Cancer)
* Pr(Dysponea | Cancer)

In [2]:
from pgmpy.factors.discrete import TabularCPD

cpd_poll = TabularCPD(variable='Pollution', variable_card=2,
                      values=[[0.9], [0.1]])
cpd_smoke = TabularCPD(variable='Smoker', variable_card=2,
                       values=[[0.3], [0.7]])
cpd_cancer = TabularCPD(variable='Cancer', variable_card=2,
                        values=[[0.03, 0.05, 0.001, 0.02],
                                [0.97, 0.95, 0.999, 0.98]],
                        evidence=['Smoker', 'Pollution'],
                        evidence_card=[2, 2])
cpd_xray = TabularCPD(variable='Xray', variable_card=2,
                      values=[[0.9, 0.2], [0.1, 0.8]],
                      evidence=['Cancer'], evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea', variable_card=2,
                      values=[[0.65, 0.3], [0.35, 0.7]],
                      evidence=['Cancer'], evidence_card=[2])

Explanation: The reason we have 2x4 array for `cpd_cancer` is that, the Cancer variable itself has 2 values, Smoker, and Pollution each have 2 possible values. The conditional table looks like:
```
Smoker     |    smoker_0           smoker 1
Pollution  |  poll_0  poll_1    poll_0   poll_1
cancer_0   |   0.03    0.05      0.001    0.01
cancer_1   |   0.97    0.95      0.999    0.98
```
Basically, each cell represents a Pr(Cancer = c | Smoker = s, Pollution = p) probability

Now that the CPDs (i.e. parameters) are specified, we add them into the model:

In [4]:
cancer_model.add_cpds(cpd_poll, cpd_smoke, cpd_cancer, cpd_xray, cpd_dysp)

# Checks if the CPDs are correctly defined for the model structure
cancer_model.check_model()

True

### Compute conditional dependencies

In [5]:
cancer_model.get_independencies()

(Pollution _|_ Smoker)
(Pollution _|_ Xray, Dyspnoea | Cancer)
(Pollution _|_ Xray | Dyspnoea, Cancer)
(Pollution _|_ Dyspnoea | Xray, Cancer)
(Pollution _|_ Xray, Dyspnoea | Smoker, Cancer)
(Pollution _|_ Xray | Smoker, Dyspnoea, Cancer)
(Pollution _|_ Dyspnoea | Xray, Smoker, Cancer)
(Smoker _|_ Pollution)
(Smoker _|_ Xray, Dyspnoea | Cancer)
(Smoker _|_ Xray, Dyspnoea | Pollution, Cancer)
(Smoker _|_ Dyspnoea | Xray, Cancer)
(Smoker _|_ Xray | Dyspnoea, Cancer)
(Smoker _|_ Dyspnoea | Pollution, Xray, Cancer)
(Smoker _|_ Xray | Pollution, Dyspnoea, Cancer)
(Xray _|_ Pollution, Dyspnoea, Smoker | Cancer)
(Xray _|_ Dyspnoea, Smoker | Pollution, Cancer)
(Xray _|_ Pollution, Smoker | Dyspnoea, Cancer)
(Xray _|_ Pollution, Dyspnoea | Smoker, Cancer)
(Xray _|_ Smoker | Pollution, Dyspnoea, Cancer)
(Xray _|_ Dyspnoea | Pollution, Smoker, Cancer)
(Xray _|_ Pollution | Smoker, Dyspnoea, Cancer)
(Dyspnoea _|_ Pollution, Xray, Smoker | Cancer)
(Dyspnoea _|_ Xray, Smoker | Pollution, Cancer)
(Dy

## Factor graphs

Factor graphs is a generalization of Bayesian Networks.

In [7]:
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor

In [29]:
G = FactorGraph()
G.add_nodes_from(["A", "B", "C"])

Add factors.
> DiscreteFactor( variables, cardinality, values, state_names={} )

`state_names` is a mapping from variable name to a list of values. 

In [30]:
phi1 = DiscreteFactor(["A", "B"], [2,2], [10, 0.4, 20, 5], state_names={"A":["a1","a2"], "B":["b1","b2"]})
phi2 = DiscreteFactor(["B", "C"], [2,2], [3, 2.4, 75, 25], state_names={"C":["c1","c2"], "B":["b1","b2"]})
G.add_factors(phi1, phi2)
# THIS IS IMPORTANT!
G.add_edges_from([('A', phi1), ('B', phi1),
                   ('B', phi2), ('C', phi2)]) 

The order matters. In the example above, `["A", "B"]` implies that the `values` correspond to:
```
   A    B     phi(A,B)
  a1    b1    10
  a1    b2    0.4
  a2    b1    20
  a2    b2    5
```
The reason `a1` is before `a2` is that I specified `{"A": ["a1","a2"]}` in the `state_names` vairable.

Printing out verifies this.

In [31]:
print(phi1)

+-------+-------+------------+
| A     | B     |   phi(A,B) |
+=======+=======+============+
| A(a1) | B(b1) |    10.0000 |
+-------+-------+------------+
| A(a1) | B(b2) |     0.4000 |
+-------+-------+------------+
| A(a2) | B(b1) |    20.0000 |
+-------+-------+------------+
| A(a2) | B(b2) |     5.0000 |
+-------+-------+------------+


In [32]:
variable_nodes = set([x for factor in G.factors for x in factor.scope()])
variable_nodes

{'A', 'B', 'C'}

In [33]:
factor_nodes = set(G.nodes()) - variable_nodes
factor_nodes

{<DiscreteFactor representing phi(A:2, B:2) at 0x7fea55442310>,
 <DiscreteFactor representing phi(B:2, C:2) at 0x7fea55442250>}

In [35]:
G.check_model()

True

### Inference

Belief propagation

In [17]:
from pgmpy.inference import BeliefPropagation

In [39]:
bp = BeliefPropagation(G)
factor = bp.query(["A","B"], evidence={"C": "c1"})

0it [00:00, ?it/s]


In [40]:
print(factor)

+-------+-------+------------+
| B     | A     |   phi(B,A) |
+=======+=======+============+
| B(b1) | A(a1) |     0.0606 |
+-------+-------+------------+
| B(b1) | A(a2) |     0.1212 |
+-------+-------+------------+
| B(b2) | A(a1) |     0.0606 |
+-------+-------+------------+
| B(b2) | A(a2) |     0.7576 |
+-------+-------+------------+


Look up probabilities in the factor

In [46]:
factor.get_value({"A":"a1", "B":"b1"})

0.0606060606060606